<a href="https://colab.research.google.com/github/jasureddy123/Jashwanth-fyp/blob/main/Topic_34_jan_26_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automated Multi-Class Classification of Brain Tumors Using Deep Learning and Explainable AI**

# **Upload Dataset**

In [ ]:
from google.colab import files

# Upload your archive.zip
uploaded = files.upload()


# **Extract ZIP**

In [ ]:
!unzip -q archive.zip -d brain_tumor_data


# **Verify Classes**

In [ ]:
import os

print("Training classes:", os.listdir("brain_tumor_data/Training"))
print("Testing classes:", os.listdir("brain_tumor_data/Testing"))


# **Load Images And Labels**

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def load_images(folder_path, image_size=(128,128)):
    X, y = [], []
    classes = sorted(os.listdir(folder_path))  # keep order consistent
    for idx, cls in enumerate(classes):
        class_path = os.path.join(folder_path, cls)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                img = load_img(img_path, target_size=image_size, color_mode='rgb')
                X.append(img_to_array(img)/255.0)  # normalize to 0-1
                y.append(idx)
            except:
                continue
    return np.array(X), np.array(y), classes

# Load training and testing sets
X_train, y_train, classes = load_images("brain_tumor_data/Training")
X_test, y_test, _ = load_images("brain_tumor_data/Testing")

print("Training set:", X_train.shape, y_train.shape)
print("Testing set:", X_test.shape, y_test.shape)
print("Classes:", classes)


# **Exploratory Data Analysis**

## **Sample Images**

In [ ]:
import matplotlib.pyplot as plt
import random

plt.figure(figsize=(12,6))
for i in range(6):
    idx = random.randint(0, len(X_train)-1)
    plt.subplot(2,3,i+1)
    plt.imshow(X_train[idx])
    plt.title(classes[y_train[idx]])
    plt.axis('off')
plt.show()


## **Class Distrubtion**

In [ ]:
import pandas as pd
import seaborn as sns

train_df = pd.DataFrame({'class': [classes[i] for i in y_train]})
plt.figure(figsize=(8,5))
sns.countplot(x='class', data=train_df)
plt.title("Training Set Class Distribution")
plt.show()


## **Image Shapes and Pixel Range**

In [ ]:
print("Sample image shape:", X_train[0].shape)
print("Pixel value range: min =", X_train.min(), "max =", X_train.max())


# **Preprocessing**

## **One Hot Encode Label**

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_cat = to_categorical(y_train, num_classes=len(classes))
y_test_cat = to_categorical(y_test, num_classes=len(classes))

print("Example one-hot label:", y_train_cat[0])
print("Shape of one-hot labels:", y_train_cat.shape)


## **Data Augmentation Pipeline**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Example: visualize augmented images
sample_batch = X_train[:10]
aug_iter = datagen.flow(sample_batch, batch_size=10)
aug_images = next(aug_iter)

plt.figure(figsize=(12,4))
for i in range(10):
    plt.subplot(2,5,i+1)
    plt.imshow(aug_images[i])
    plt.axis('off')
plt.suptitle("Sample Augmented Images")
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout

def build_model(base_model_name, num_classes):
    # Load the base model with pre-trained ImageNet weights
    if base_model_name == 'VGG16':
        base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    elif base_model_name == 'ResNet50':
        base = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    elif base_model_name == 'EfficientNetB0':
        base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    # Freeze the base layers so we only train the new head
    base.trainable = False

    # Add custom classification head
    x = base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## **Model Traning**

In [ ]:
models_to_test = ['VGG16', 'ResNet50', 'EfficientNetB0']
histories = {}
trained_models = {}

for name in models_to_test:
    print(f"\n--- Training {name} ---")
    model = build_model(name, len(classes))

    # Train using the augmented datagen from your previous code
    history = model.fit(
        datagen.flow(X_train, y_train_cat, batch_size=32),
        validation_data=(X_test, y_test_cat),
        epochs=10, # Start with 10 for comparison
        verbose=1
    )

    histories[name] = history
    trained_models[name] = model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, accuracy_score, confusion_matrix,
    roc_curve, auc, precision_score, recall_score, f1_score
)
from sklearn.preprocessing import label_binarize

# 1. Setup for Multi-class ROC
n_classes = len(classes)
y_test_bin = label_binarize(y_test, classes=range(n_classes))

for name in models_to_test:
    print(f"\n" + "="*40)
    print(f" EVALUATING MODEL: {name}")
    print("="*40)

    model = trained_models[name]

    # Get predictions (probabilities and hard classes)
    y_pred_probs = model.predict(X_test)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # --- [A] TEXT METRICS ---
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=classes))

    # --- [B] CONFUSION MATRIX ---
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(12, 5))

    # Subplot 1: Confusion Matrix
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.title(f'Confusion Matrix: {name}')
    plt.ylabel('Actual Label')
    plt.xlabel('Predicted Label')

    # --- [C] ROC CURVE ---
    plt.subplot(1, 2, 2)
    colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']
    for i in range(n_classes):
        fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_pred_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, color=colors[i], lw=2,
                 label=f'{classes[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve: {name}')
    plt.legend(loc="lower right")

    plt.tight_layout()
    plt.show()

# **How Advanced Augmention can affect the result**

In [ ]:
# Select the best model architecture from previous steps (e.g., EfficientNetB0)
baseline_model = build_model('EfficientNetB0', len(classes))

print("--- Training Baseline Model (Raw Data) ---")
# Using validation_split to ensure we have a testable baseline
baseline_history = baseline_model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    batch_size=32,
    epochs=15,
    verbose=1
)

In [ ]:
augmented_model = build_model('VGG16', len(classes))

print("\n--- Training Augmented Model (Advanced Techniques) ---")
# Using the datagen.flow method for real-time augmentation
aug_history = augmented_model.fit(
    datagen.flow(X_train, y_train_cat, batch_size=32),
    validation_data=(X_test, y_test_cat),
    epochs=15,
    verbose=1
)

In [ ]:
from sklearn.metrics import recall_score
import pandas as pd

def get_recall_per_class(model, X, y_true, class_names):
    y_pred = np.argmax(model.predict(X), axis=1)
    # Calculate recall for each class specifically
    recalls = recall_score(y_true, y_pred, average=None)
    return dict(zip(class_names, recalls))

# Get metrics
baseline_recalls = get_recall_per_class(baseline_model, X_test, y_test, classes)
augmented_recalls = get_recall_per_class(augmented_model, X_test, y_test, classes)

# Create comparison table
comparison_df = pd.DataFrame({
    'Class': classes,
    'Baseline Recall (Raw)': [baseline_recalls[c] for c in classes],
    'Augmented Recall (Improved)': [augmented_recalls[c] for c in classes]
})

comparison_df['Improvement (%)'] = (comparison_df['Augmented Recall (Improved)'] - comparison_df['Baseline Recall (Raw)']) * 100
print(comparison_df)

In [ ]:
plt.figure(figsize=(10, 6))
x = np.arange(len(classes))
width = 0.35

plt.bar(x - width/2, comparison_df['Baseline Recall (Raw)'], width, label='Raw Data', color='gray')
plt.bar(x + width/2, comparison_df['Augmented Recall (Improved)'], width, label='With Augmentation', color='skyblue')

plt.xlabel('Tumor Class')
plt.ylabel('Recall (Sensitivity)')
plt.title('Impact of Augmentation on Model Sensitivity')
plt.xticks(x, classes)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# **Visualization with GradCam**

In [ ]:
import tensorflow as tf
import cv2
import numpy as np

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 1. Create a model that maps the input image to the activations of the last conv layer
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # 2. Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # 3. This is the gradient of the output neuron wrt the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 4. Vector of mean intensity of the gradients over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 5. Multiply each channel in the feature map array by "how important this channel is"
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 6. Normalize the heatmap between 0 & 1 for visualization
    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.math.reduce_max(heatmap)
    if max_val == 0:
        # If max_val is 0, all activations were non-positive for the target class.
        # This means no relevant area was found. Return a zero heatmap of the same shape.
        return tf.zeros_like(heatmap).numpy()
    else:
        heatmap /= max_val
    return heatmap.numpy()

def display_gradcam(img, heatmap, alpha=0.4):
    # Resize heatmap to match image size
    # cv2.resize expects (width, height)
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)

    # Handle potential NaNs from make_gradcam_heatmap if max_val was zero
    heatmap_resized = np.nan_to_num(heatmap_resized)

    # Normalize heatmap for visualization (0-1 range) to ensure proper colormap scaling
    max_val_resized = np.max(heatmap_resized)
    if max_val_resized == 0: # If all values are zero, no heat to display, return original image.
        return tf.keras.preprocessing.image.array_to_img(np.uint8(255 * img))
    heatmap_normalized = heatmap_resized / max_val_resized

    # Rescale heatmap to 0-255 for colormap application
    heatmap_colored = np.uint8(255 * heatmap_normalized)

    # Apply colormap
    jet = cv2.applyColorMap(heatmap_colored, cv2.COLORMAP_JET)
    jet = cv2.cvtColor(jet, cv2.COLOR_BGR2RGB) # OpenCV reads BGR, convert to RGB

    # Prepare original image for blending
    img_display = np.uint8(255 * img) # Convert original image (0-1 float) to (0-255 uint8)

    # Superimpose the heatmap on original image using alpha blending
    superimposed_img = jet * alpha + img_display * (1 - alpha)
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

    # Convert to PIL Image
    return tf.keras.preprocessing.image.array_to_img(superimposed_img)

# --- EXECUTION ---
# Note: For EfficientNetB0, the last conv layer is usually 'top_activation'
# For VGG16, it is 'block5_conv3'
model_to_explain = trained_models['EfficientNetB0']
sample_img = np.expand_dims(X_test[10], axis=0) # Pick a test image

heatmap = make_gradcam_heatmap(sample_img, model_to_explain, 'top_activation')
result_img = display_gradcam(X_test[10], heatmap)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1); plt.imshow(X_test[10]); plt.title("Original MRI")
plt.subplot(1, 2, 2); plt.imshow(result_img); plt.title("Grad-CAM Localization")
plt.show()